# Computation of $\Delta\Gamma_d$ from effective lifetime measurements of $B^0 \to$ CP-eigenstate decays

In [32]:
import numpy as np
from iminuit import Minuit
from uncertainties import ufloat, correlated_values

Relations between the effective lifetimes of $B^0\to J/\psi K_S$ and $B^0\to J/\psi K_S$ and the parameters $\Gamma$, $\Delta\Gamma$ and CKM-angle $\beta$. From arXiv:1109.5115

In [33]:
def tauKst(G, DG):
    return 1/G*1/(1 - (DG/(2*G))**2)*(1 + (DG/(2*G))**2)
def tauKS(G, DG, beta):
    return 1/G*1/(1 - (DG/(2*G))**2)*(1 + 2*np.cos(2*beta)*(DG/(2*G)) + (DG/(2*G))**2)/(1+np.cos(2*beta)*(DG/(2*G)))

Define the function to be minimised. Need choices here about the systematic uncertainties that are correlated/uncorrelated.

In [34]:
def fcn_lhcb(G, DG, beta):  
    # LHCb 1/fb results
    tKst  = 1.5239
    dtKst_stat        = 0.0063
    dtKst_syst_total  = 0.0039
    dtKst_syst_uncorr = np.sqrt(2.3**2 + 1.8**2 + 1.2**2 + 0.2**2)/1000.
    dtKst_syst_corr   = np.sqrt(2.3**2 + 0.2**2 + 0.3**2)/1000.
    
    tKS  = 1.4987
    dtKS_stat        = 0.0129
    dtKS_syst_total  = 0.0048
    dtKS_syst_uncorr = np.sqrt(2.9**2 + 2.1**2 + 2.0**2 + 2.2**2)/1000.
    dtKS_syst_corr   = np.sqrt(0.9**2 + 0.4**2 + 0.3**2 + 1.1**2 + 0.3**2)/1000.
    
    rho = 1.
    
    # Constraint on CKM beta
    beta_constraint = 21.5/180*np.pi
    dbeta = 0.8/180*np.pi

    diff = np.matrix([tKst - tauKst(G,DG), tKS - tauKS(G, DG, beta), beta - beta_constraint])
    covMatrixStat = np.matrix([[dtKst_stat*dtKst_stat,0,0],[0,dtKS_stat*dtKS_stat,0],[0,0,dbeta*dbeta]])
    covMatrixSyst = np.matrix([ [dtKst_syst_uncorr*dtKst_syst_uncorr ,dtKst_syst_corr*dtKS_syst_corr*rho,0]
                               ,[dtKst_syst_corr  *dtKS_syst_corr*rho,dtKS_syst_uncorr*dtKS_syst_uncorr,0]
                               ,[0,0,0]])
    # Turn off either the stat or syst uncertainties to check how they impact the combination
    #covMatrixStat = np.matrix([[0,0,0],[0,0,0],[0,0,dbeta*dbeta]])
    #covMatrixSyst = np.matrix([[0,0,0],[0,0,0],[0,0,0]])
    chisq = diff*(covMatrixStat + covMatrixSyst).I*diff.transpose()
    return chisq[0,0]

In [50]:
def fcn_cms(G, DG, beta):
    c_light = 299.792458
    
    # CMS results
    tKst = 453.0/c_light
    dtKst_stat        = 1.6/c_light
    dtKst_syst_total  = 1.8/c_light
    dtKst_syst_uncorr = np.sqrt(1.1**2 + 0.3**2)/c_light
    dtKst_syst_corr   = np.sqrt(0.3**2 + 0.1**2 + 1.3**2)/c_light
    
    tKS  = 457.8/c_light
    dtKS_stat        = 2.7/c_light
    dtKS_syst_total  = 2.8/c_light
    dtKS_syst_uncorr = 2.4/c_light
    dtKS_syst_corr   = np.sqrt(0.4**2 + 0.1**2 + 1.3**2)/c_light
    
    # Assume fully correlated
    rho = 1.
    
    # Constraint on CKM beta
    beta_constraint = 21.9/180*np.pi
    dbeta = 0.7/180*np.pi

    diff = np.matrix([tKst - tauKst(G,DG), tKS - tauKS(G, DG, beta), beta - beta_constraint])
    covMatrixStat = np.matrix([[dtKst_stat*dtKst_stat,0,0],[0,dtKS_stat*dtKS_stat,0],[0,0,dbeta*dbeta]])
    covMatrixSyst = np.matrix([ [dtKst_syst_uncorr*dtKst_syst_uncorr,dtKst_syst_corr*dtKS_syst_corr*rho,0]
                               ,[dtKst_syst_corr*dtKS_syst_corr*rho,dtKS_syst_uncorr*dtKS_syst_uncorr,0]
                               ,[0,0,0]])
    # Turn off either the stat or syst uncertainties to check how they impact the combination
    #covMatrixSyst = np.matrix([[0,0,0],[0,0,0],[0,0,0]])
    #covMatrixStat = np.matrix([[0,0,0],[0,0,0],[0,0,0]])
    chisq = diff*(covMatrixStat + covMatrixSyst).I*diff.transpose()
    return chisq[0,0]

Do the minimisation

In [51]:
m = Minuit(fcn_cms, G = 0.66, DG = 0.02)
m.migrad()

/usr/local/lib/swan/ipykernel/__main__.py:1: InitialParamWarning: errordef is not given. Default to 1.
  if __name__ == '__main__':
/usr/local/lib/swan/ipykernel/__main__.py:1: InitialParamWarning: Parameter G is floating but does not have initial step size. Assume 1.
  if __name__ == '__main__':
/usr/local/lib/swan/ipykernel/__main__.py:1: InitialParamWarning: Parameter DG is floating but does not have initial step size. Assume 1.
  if __name__ == '__main__':
/usr/local/lib/swan/ipykernel/__main__.py:1: InitialParamWarning: Parameter beta does not have initial value. Assume 0.
  if __name__ == '__main__':
/usr/local/lib/swan/ipykernel/__main__.py:1: InitialParamWarning: Parameter beta is floating but does not have initial step size. Assume 1.
  if __name__ == '__main__':


FCN = 3.9865021644996643e-07 
 TOTAL NCALL = 66 
 NCALLS = 66 
 
 
 EDM = 3.9865341068533205e-07 
 GOAL EDM = 1e-05 
 
 UP = 1.0 
 
 
 
 
 
 Valid 
 Valid Param 
 Accurate Covar 
 PosDef 
 Made PosDef 
 
 
 True 
 True 
 True 
 True 
 False 
 
 
 Hesse Fail 
 HasCov 
 Above EDM 
 
 Reach calllim 
 
 
 False 
 True 
 False 
 
 False

+,Name,Value,Parab Error,Minos Error-,Minos Error+,Limit-,Limit+,FIXED
1,G,0.662084,0.00303201,0,0,,,
2,DG,0.019648,0.0151557,0,0,,,
3,beta,0.382224,0.0122173,0,0,,,


({'fval': 3.9865021644996643e-07, 'edm': 3.9865341068533205e-07, 'nfcn': 66, 'up': 1.0, 'is_valid': True, 'has_valid_parameters': True, 'has_accurate_covar': True, 'has_posdef_covar': True, 'has_made_posdef_covar': False, 'hesse_failed': False, 'has_covariance': True, 'is_above_max_edm': False, 'has_reached_call_limit': False},
 [{'number': 0, 'name': 'G', 'value': 0.6620844689090644, 'error': 0.00303200535344773, 'is_const': False, 'is_fixed': False, 'has_limits': False, 'has_lower_limit': False, 'has_upper_limit': False, 'lower_limit': 0.0, 'upper_limit': 0.0},
  {'number': 1, 'name': 'DG', 'value': 0.019647999460563417, 'error': 0.015155670499890245, 'is_const': False, 'is_fixed': False, 'has_limits': False, 'has_lower_limit': False, 'has_upper_limit': False, 'lower_limit': 0.0, 'upper_limit': 0.0},
  {'number': 2, 'name': 'beta', 'value': 0.38222394796865544, 'error': 0.012217301384286706, 'is_const': False, 'is_fixed': False, 'has_limits': False, 'has_lower_limit': False, 'has_upp

Correlation between $\Gamma$ and $\Delta\Gamma$

In [52]:
rho = m.covariance[('G', 'DG')]/m.errors['G']/m.errors['DG']
print(rho)

0.42711747016315316


Check that $\beta$ has not changed

In [53]:
print("beta =",ufloat(m.values['beta']*180/np.pi,m.errors['beta']*180/np.pi))

beta = 21.9+/-0.7


Compute $\Delta\Gamma/\Gamma$. Use uncertainties python package to combine the uncertainies (accounting for the correlation between G and DG has no impact here since the uncertainty on DG dominates).

In [54]:
(G, DG) = correlated_values([m.values['G'], m.values['DG']], [(m.errors['G']*m.errors['G'],rho*m.errors['G']*m.errors['DG']),(rho*m.errors['G']*m.errors['DG'],m.errors['DG']*m.errors['DG'])])
print("G    =",G)
print("DG   =",DG)
print("DG/G =",DG/G)

G    = 0.6621+/-0.0030
DG   = 0.020+/-0.015
DG/G = 0.030+/-0.023


The published CMS results are
\begin{equation}
\Gamma              = 0.662 \pm 0.003\ (stat) \pm 0.003\ (syst) = 0.662 \pm 0.004\\
\Delta\Gamma        = 0.023 \pm 0.015\ (stat) \pm 0.016\ (syst) = 0.023 \pm 0.022\\
\Delta\Gamma/\Gamma = 0.034 \pm 0.023\ (stat) \pm 0.024\ (syst) = 0.034 \pm 0.033
\end{equation}

Why are the central values and uncertainties different for $\Delta\Gamma$ (and, by extension, $\Delta\Gamma/\Gamma$)? Can play with the assumptions about which uncertainties are correlated to see what difference it makes.

Note that if I turn off all systematic uncertainties I get
\begin{equation}
\Gamma              = 0.6621 \pm 0.0026\ (stat)\\
\Delta\Gamma        = 0.020 \pm 0.013\ (stat)\\
\Delta\Gamma/\Gamma = 0.030 \pm 0.020\ (stat)
\end{equation}